In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Step 1. Load dataset ```covid19_us_summary.csv```

```covid19_us_summary.csv``` is created by joining ```bigquery-public-data.covid19_google_mobility.mobility_report``` and ```bigquery-public-data.covid19_usafacts.summary```. It contains the time series state-level data of:
* confirmed_cases 
* deaths
* retail_and_recreation (percent_change_from_baseline)
* grocery_and_pharmacy (percent_change_from_baseline)
* parks (percent_change_from_baseline)
* transit_stations (percent_change_from_baseline)
* workplaces (percent_change_from_baseline)
* residential (percent_change_from_baseline)

In [2]:
df_us = pd.read_csv('covid19_us_summary.csv')
df_us.head(3)

,Unnamed: 0,a_state,a_date,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,b_state,b_date,confirmed_cases_state,deaths_state
0,1,AL,2020-02-15,5,2,39.0,7,2,-1,AL,2020-02-15,0,0
1,2,AL,2020-02-16,0,-2,-7.0,3,-1,1,AL,2020-02-16,0,0
2,3,AL,2020-02-17,3,0,17.0,7,-17,4,AL,2020-02-17,0,0


In [3]:
# drop duplicated columns "b_state" and "b_date", and rename columns name.
df_us=df_us.drop(['b_state','b_date'],axis=1)
df_us=df_us.rename(columns={"a_state": "state", "a_date": "date", 
                      "confirmed_cases_state": "confirmed_cases", "deaths_state": "deaths"})

In [4]:
print(df_us.shape)
print(df_us.dtypes)
#df_us.isnull().sum()

(5814, 11)
Unnamed: 0                 int64
state                     object
date                      object
retail_and_recreation      int64
grocery_and_pharmacy       int64
parks                    float64
transit_stations           int64
workplaces                 int64
residential                int64
confirmed_cases            int64
deaths                     int64
dtype: object


In [5]:
df_us["state"].value_counts()

DC    114
WA    114
MN    114
DE    114
VT    114
WY    114
NV    114
OR    114
WI    114
CA    114
TN    114
AZ    114
UT    114
MS    114
NJ    114
AR    114
CT    114
MI    114
MT    114
KS    114
IN    114
ID    114
IA    114
HI    114
NC    114
NM    114
NE    114
KY    114
NH    114
OK    114
OH    114
AL    114
WV    114
FL    114
MA    114
MO    114
ND    114
TX    114
AK    114
VA    114
GA    114
CO    114
RI    114
LA    114
ME    114
MD    114
PA    114
SD    114
SC    114
IL    114
NY    114
Name: state, dtype: int64

In [6]:
# 51 states in total
df_us["state"].unique()

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'], dtype=object)

In [7]:
df_us.describe()

,Unnamed: 0,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,confirmed_cases,deaths
count,5814.000000,5814.000000,5814.000000,5814.000000,5814.000000,5814.000000,5814.000000,5814.000000,5814.000000
mean,2907.500000,-20.804266,-2.714826,31.239346,-24.139663,-27.986928,10.464568,13502.932577,759.929137
std,1678.501564,20.771848,12.653647,54.731013,23.995565,19.929952,8.042680,37716.769020,2784.861869
min,1.000000,-77.000000,-62.000000,-77.000000,-81.000000,-78.000000,-5.000000,0.000000,0.000000
25%,1454.250000,-37.000000,-11.000000,-3.000000,-44.000000,-43.000000,3.000000,19.000000,0.000000
50%,2907.500000,-23.000000,-2.000000,20.000000,-25.000000,-33.000000,11.000000,1524.000000,39.000000
75%,4360.750000,-1.000000,6.000000,55.000000,-1.000000,-8.000000,17.000000,10251.000000,377.750000
max,5814.000000,24.000000,48.000000,360.000000,41.000000,18.000000,33.000000,378166.000000,30170.000000


# Step 2. Load dataset ```COVID-19 US state policy database 6_10_2020.csv```

In [8]:
df_policy = pd.read_excel('COVID-19 US state policy database 6_10_2020.xlsx')
df_policy.head()

,STATE,POSTCODE,STEMERG,CLSCHOOL,CLDAYCR,CLNURSHM,STAYHOME,END_STHM,CLBSNS,END_BSNS,...,PDSKLV,MEDEXP,POPDEN18,POP18,SQML,HMLS19,UNEMP18,POV18,RISKCOV,DEATH18
0,State,Postcode,State of emergency,Date closed K-12 schools,Closed day cares,Date banned visitors to nursing homes,Stay at home/ shelter in place,End/relax stay at home/shelter in place,Closed non-essential businesses,Began to reopen businesses,...,Paid sick leave,Medicaid Expansion,Population density per square miles,Population 2018,Square Miles,Number Homeless (2019),Percent Unemployed (2018),Percent living under the federal poverty line ...,Percent at risk for serious illness due to COVID,All-cause deaths 2018
1,category,postcode,emergency,school_closure,day_care_closure,nursing_home_visit_ban,shelter,shelter,business_closure,business_closure,...,paid_sick_leave,medicaid_expansion,population_density,population,area,homeless,unemployment,poverty,at_risk,all_cause_deaths
2,type,note,start,start,start,start,start,end,start,end,...,attribute,attribute,quantity,quantity,quantity,quantity,quantity,quantity,quantity,quantity
3,unit,text,date,date,date,date,date,date,date,date,...,flag,flag,people/sq mi,people,sq mi,people,percent,percent,percent,people/year
4,Alabama,AL,2020-03-13 00:00:00,2020-03-19 00:00:00,2020-03-20 00:00:00,2020-03-19 00:00:00,2020-04-04 00:00:00,2020-04-30 00:00:00,2020-03-28 00:00:00,2020-04-30 00:00:00,...,0,0,93.24,4887871,52420,3261,5.6,16.8,43.1,54352


In [9]:
list(df_policy.columns) 

['STATE',
 'POSTCODE',
 'STEMERG',
 'CLSCHOOL',
 'CLDAYCR',
 'CLNURSHM',
 'STAYHOME',
 'END_STHM',
 'CLBSNS',
 'END_BSNS',
 'RELIGEX',
 'FM_ALL',
 'FM_EMP',
 'ALCOPEN',
 'GUNOPEN',
 'CLREST',
 'ENDREST',
 'RSTOUTDR',
 'CLGYM',
 'ENDGYM',
 'CLMOVIE',
 'END_MOV',
 'EVICINTN',
 'EVICENF',
 'RNTGP',
 'UTILSO',
 'MORGFR',
 'SNAPALLO',
 'SNAPEBT',
 'SNAPSUSP',
 'MED1135W',
 'ACAENROL',
 'PREVTLHL',
 'TLHLAUD',
 'TLHLMED',
 'RACETEST',
 'RACECASE',
 'RACEHOSP',
 'RACEDEAT',
 'ELECPRCR',
 'ENDELECP',
 'WTPRD',
 'WV_WTPRD',
 'WV_WKSR',
 'UIQUAR',
 'UICLDCR',
 'UIEXTND',
 'UIMAXAMT',
 'UIMAXEXT',
 'UIMAXDUR',
 'UIMAXCAR',
 'LMABRN',
 'TLHlBUPR',
 'EXTOPFL',
 'HMDLVOP',
 'TLHLCL24',
 'EXCEMORP',
 'WVDEAREQ',
 'PDSKLV',
 'MEDEXP',
 'POPDEN18',
 'POP18',
 'SQML',
 'HMLS19',
 'UNEMP18',
 'POV18',
 'RISKCOV',
 'DEATH18']

# Step 3. Create IL sub-dataset for time series modeling

In [10]:
df_il=df_us[df_us['state']=='IL']

In [11]:
df_il.head()

,Unnamed: 0,state,date,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,confirmed_cases,deaths
1482,1483,IL,2020-02-15,4,3,10.0,3,3,-1,2,0
1483,1484,IL,2020-02-16,8,2,29.0,3,2,-1,2,0
1484,1485,IL,2020-02-17,2,-3,-14.0,-13,-23,6,2,0
1485,1486,IL,2020-02-18,1,-1,5.0,2,1,0,2,0
1486,1487,IL,2020-02-19,1,-1,3.0,2,1,0,2,0


In [12]:
df_il.describe()

,Unnamed: 0,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,confirmed_cases,deaths
count,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000
mean,1539.500000,-25.991228,-2.719298,17.359649,-32.973684,-31.008772,12.096491,39698.342105,1728.903509
std,33.052988,20.737506,12.710882,41.874470,22.138526,21.491095,8.694388,44553.063825,2019.122126
min,1483.000000,-64.000000,-34.000000,-48.000000,-56.000000,-76.000000,-3.000000,2.000000,0.000000
25%,1511.250000,-41.750000,-11.750000,-11.750000,-50.000000,-49.750000,3.000000,70.250000,0.000000
50%,1539.500000,-33.000000,-3.000000,10.000000,-42.500000,-39.000000,15.000000,20015.000000,698.500000
75%,1567.750000,-7.250000,3.000000,35.000000,-13.000000,-6.500000,20.000000,77320.000000,3390.500000
max,1596.000000,17.000000,43.000000,165.000000,13.000000,7.000000,24.000000,127747.000000,5904.000000


In [13]:
def policy(date, policy_date):
    if policy_date == 0:
        return 0
    elif date >= policy_date:
        return 1
    else:
        return 0  

In [14]:
df_policy.loc[df_policy.POSTCODE=='IL']

,STATE,POSTCODE,STEMERG,CLSCHOOL,CLDAYCR,CLNURSHM,STAYHOME,END_STHM,CLBSNS,END_BSNS,...,PDSKLV,MEDEXP,POPDEN18,POP18,SQML,HMLS19,UNEMP18,POV18,RISKCOV,DEATH18
17,Illinois,IL,2020-03-09 00:00:00,2020-03-17 00:00:00,2020-03-23 00:00:00,0,2020-03-21 00:00:00,2020-05-29 00:00:00,2020-03-21 00:00:00,2020-05-29 00:00:00,...,0,1,220,12741080,57914,10199,5.5,12.1,36.2,110022


In [15]:
p = ['STEMERG', 'CLSCHOOL', 'CLDAYCR', 'STAYHOME', 'CLBSNS', 'FM_ALL', 'FM_EMP', 'CLREST',
                    'CLGYM', 'CLMOVIE', 'CLNURSHM', 'EVICINTN', 'EVICENF', 'END_STHM', 'END_BSNS', 
                    'ENDREST', 'ENDGYM', 'END_MOV']

In [16]:
for i in range(len(p)):
    if df_policy.loc[df_policy.POSTCODE=='IL',p[i]].values[0] == 0:
        df_il[p[i]] = 0
    else:
        policy_date = str(df_policy.loc[df_policy.POSTCODE=='IL',p[i]].dt.date.values[0])
        df_il[p[i]] = df_il.apply(lambda x: policy(x['date'],policy_date),axis=1)

In [17]:
df_il.head()

,Unnamed: 0,state,date,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,confirmed_cases,...,CLGYM,CLMOVIE,CLNURSHM,EVICINTN,EVICENF,END_STHM,END_BSNS,ENDREST,ENDGYM,END_MOV
1482,1483,IL,2020-02-15,4,3,10.0,3,3,-1,2,...,0,0,0,0,0,0,0,0,0,0
1483,1484,IL,2020-02-16,8,2,29.0,3,2,-1,2,...,0,0,0,0,0,0,0,0,0,0
1484,1485,IL,2020-02-17,2,-3,-14.0,-13,-23,6,2,...,0,0,0,0,0,0,0,0,0,0
1485,1486,IL,2020-02-18,1,-1,5.0,2,1,0,2,...,0,0,0,0,0,0,0,0,0,0
1486,1487,IL,2020-02-19,1,-1,3.0,2,1,0,2,...,0,0,0,0,0,0,0,0,0,0


# Step 4. Calculate PolicyScores, which will be used as input for our predictive models on mobility)
### Policy Scores = dot product of policies weights and policies dummy tables. ```policies weights``` are the outputs of ```PolicyClustering.ipynb```.

In [18]:
# PolicyScore for retail mobility
policies_retail = pd.read_csv('policies_retail.csv')
policies_retail.head()

,Unnamed: 0,k_label,AL,AK,AZ,AR,CA,CO,CT,DE,...,TN,TX,UT,VT,VA,WA,WV,WI,WY,POLICIES
0,0,0,-0.130040,-0.037708,-0.017445,-0.031850,-0.090165,-0.095043,-0.080998,-0.116391,...,-0.046100,-0.084640,-0.005212,-0.132759,-0.010886,-0.138855,-0.069246,-0.083222,-0.059877,STEMERG
1,1,2,-0.157347,-0.265821,-0.225091,-0.163388,-0.217445,-0.240451,-0.241932,-0.219397,...,-0.144764,-0.127537,-0.112230,-0.234464,-0.101730,-0.211800,-0.225014,-0.262630,-0.306383,CLSCHOOL
2,2,0,-0.130040,-0.037708,-0.017445,-0.031850,-0.090165,-0.095043,-0.080998,-0.116391,...,-0.046100,-0.084640,-0.005212,-0.132759,-0.010886,-0.138855,-0.069246,-0.083222,-0.059877,CLDAYCR
3,3,0,-0.130040,-0.037708,-0.017445,-0.031850,-0.090165,-0.095043,-0.080998,-0.116391,...,-0.046100,-0.084640,-0.005212,-0.132759,-0.010886,-0.138855,-0.069246,-0.083222,-0.059877,STAYHOME
4,4,0,-0.130040,-0.037708,-0.017445,-0.031850,-0.090165,-0.095043,-0.080998,-0.116391,...,-0.046100,-0.084640,-0.005212,-0.132759,-0.010886,-0.138855,-0.069246,-0.083222,-0.059877,CLBSNS


In [19]:
policies_retail[['POLICIES','IL']].head()

,POLICIES,IL
0,STEMERG,-0.153976
1,CLSCHOOL,-0.305791
2,CLDAYCR,-0.153976
3,STAYHOME,-0.153976
4,CLBSNS,-0.153976


In [20]:
df_il[p].head()

,STEMERG,CLSCHOOL,CLDAYCR,STAYHOME,CLBSNS,FM_ALL,FM_EMP,CLREST,CLGYM,CLMOVIE,CLNURSHM,EVICINTN,EVICENF,END_STHM,END_BSNS,ENDREST,ENDGYM,END_MOV
1482,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1483,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1484,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1485,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1486,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
df_il['policyScore_retail'] = df_il[p].dot(policies_retail[['POLICIES','IL']].set_index('POLICIES')) *100
df_il['policyScore_retail'].value_counts()

-199.304279    39
-192.005616    28
 0.000000      23
-177.408288     8
-15.397595      7
-76.555747      4
-183.906684     2
-177.408288     2
-45.976671      1
Name: policyScore_retail, dtype: int64

In [22]:
# PolicyScore for grocery mobility
policies_grocery = pd.read_csv('policies_grocery.csv')
df_il['policyScore_grocery'] = df_il[p].dot(policies_grocery[['POLICIES','IL']].set_index('POLICIES'))*100
df_il['policyScore_grocery'].value_counts()

-149.982576    39
-155.972190    28
 0.000000      23
-167.951418     8
-2.994807       7
-49.994192      4
-167.951418     2
-146.987769     2
-26.494500      1
Name: policyScore_grocery, dtype: int64

In [23]:
# PolicyScore for parks mobility
policies_parks = pd.read_csv('policies_parks.csv')
df_il['policyScore_parks'] = df_il[p].dot(policies_parks[['POLICIES','IL']].set_index('POLICIES'))*100
df_il['policyScore_parks'].value_counts()

-12.127021    39
 4.809913     28
 0.000000     23
 36.190559    10
 8.468467      7
-9.371818      4
-20.595489     2
-0.451676      1
Name: policyScore_parks, dtype: int64

In [24]:
# PolicyScore for transit mobility
policies_transit = pd.read_csv('policies_transit.csv')
df_il['policyScore_transit'] = df_il[p].dot(policies_transit[['POLICIES','IL']].set_index('POLICIES'))*100
df_il['policyScore_transit'].value_counts()

-186.201377    39
-162.798052    28
 0.000000      23
-115.991403    10
-6.887337       7
-59.819438      4
-179.314039     2
-33.353388      1
Name: policyScore_transit, dtype: int64

In [25]:
# PolicyScore for workplaces mobility
policies_workplaces = pd.read_csv('policies_workplaces.csv')
df_il['policyScore_workplaces'] = df_il[p].dot(policies_workplaces[['POLICIES','IL']].set_index('POLICIES'))*100
df_il['policyScore_workplaces'].value_counts()

-144.134474    39
-105.160937    28
 0.000000      23
-27.213863      8
-10.680121      7
-56.047055      4
-27.213863      2
-133.454352     2
-33.363588      1
Name: policyScore_workplaces, dtype: int64

In [26]:
# PolicyScore for residential mobility
policies_residential = pd.read_csv('policies_residential.csv')
df_il['policyScore_residential'] = df_il[p].dot(policies_residential[['POLICIES','IL']].set_index('POLICIES'))*100
df_il['policyScore_residential'].value_counts()

46.702530    39
41.701145    28
0.000000     23
31.698374     8
3.139055      7
18.642682     4
43.563475     2
31.698374     2
10.890869     1
Name: policyScore_residential, dtype: int64

In [27]:
df_il.columns

Index(['Unnamed: 0', 'state', 'date', 'retail_and_recreation',
       'grocery_and_pharmacy', 'parks', 'transit_stations', 'workplaces',
       'residential', 'confirmed_cases', 'deaths', 'STEMERG', 'CLSCHOOL',
       'CLDAYCR', 'STAYHOME', 'CLBSNS', 'FM_ALL', 'FM_EMP', 'CLREST', 'CLGYM',
       'CLMOVIE', 'CLNURSHM', 'EVICINTN', 'EVICENF', 'END_STHM', 'END_BSNS',
       'ENDREST', 'ENDGYM', 'END_MOV', 'policyScore_retail',
       'policyScore_grocery', 'policyScore_parks', 'policyScore_transit',
       'policyScore_workplaces', 'policyScore_residential'],
      dtype='object')

In [28]:
df_il_new = df_il[['date', 'retail_and_recreation', 'grocery_and_pharmacy', 'parks', 'transit_stations', 'workplaces',
       'residential', 'confirmed_cases', 'deaths', 'policyScore_retail', 'policyScore_grocery', 'policyScore_parks',
      'policyScore_transit', 'policyScore_workplaces', 'policyScore_residential']]

In [29]:
df_il_new.head()

,date,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,confirmed_cases,deaths,policyScore_retail,policyScore_grocery,policyScore_parks,policyScore_transit,policyScore_workplaces,policyScore_residential
1482,2020-02-15,4,3,10.0,3,3,-1,2,0,0.0,0.0,0.0,0.0,0.0,0.0
1483,2020-02-16,8,2,29.0,3,2,-1,2,0,0.0,0.0,0.0,0.0,0.0,0.0
1484,2020-02-17,2,-3,-14.0,-13,-23,6,2,0,0.0,0.0,0.0,0.0,0.0,0.0
1485,2020-02-18,1,-1,5.0,2,1,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0
1486,2020-02-19,1,-1,3.0,2,1,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
df_il_new.to_csv('df_il.csv')